<a href="https://colab.research.google.com/github/wujj0326/COVID_Risk_Competition/blob/master/Get_risk_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LA County Risk Score

## import risk data

In [175]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [176]:
risk = pd.read_csv('https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/risk_confirmed.csv')

In [177]:
risk.head()

,City,confirmed_cases,death_cases,pop2010,avg_traffic,avg_asthma,avg_cardiovascular,avg_poverty,avg_elderly,pop_dens
0,Acton,11,1,53654,1058.69,73.62,11.00,70.81,5.24,120974.0
1,Agoura Hills,35,1,60009,1023.45,77.60,10.95,72.49,5.45,177875.5
2,Alhambra,226,8,112060,1816.86,47.43,7.17,46.00,10.40,551582.0
3,Altadena,157,5,69531,1379.59,34.64,5.34,55.34,10.67,864902.9
4,Arcadia,104,7,70416,1305.56,49.61,6.12,73.43,8.78,755363.9


In [178]:
# normalize the scale by using minmaxscaler
scale_risk = MinMaxScaler(feature_range=(0, 1))
features = scale_risk.fit_transform(risk.iloc[:,3:-1])

#  vulnerable factors related to death cases

# •       elderly

# •       asthma

# •       cardiovascular

#  vulnearble factors related to infected cases

# •       poverty: the higher the value the poorer the area

# •       traffic

# •       population

confirm_feat = np.sum(features[:,np.r_[0,1,4]],axis=1)
death_feat = np.sum(features[:,np.r_[2,3,5]],axis=1)

In [179]:
# apply sigmoid function to confirm feature and death feature
sig_confirmed = 1/(1 + np.exp(confirm_feat)) 
sig_death = 1/(1 + np.exp(death_feat)) 

In [180]:
risk['confirmed_score'] = sig_confirmed
risk['death_score'] = sig_death

In [181]:
risk_score = risk.iloc[:,np.r_[0,-2,-1]]

## import the predicted data

In [182]:
case_fit = pd.read_csv('https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/cases_prediction.csv')

In [183]:
case_fit

,time,positive,positive_low,positive_high
0,7/29/2020,181121.1035,179840.8907,182761.2016
1,7/30/2020,183586.7964,182180.9785,185600.7594
2,7/31/2020,186078.0061,184542.7771,188478.2714
3,8/1/2020,188595.2395,186926.7146,191394.5058
4,8/2/2020,191139.0128,189333.2267,194350.2463
5,8/3/2020,193709.8516,191762.7570,197346.2921
6,8/4/2020,196308.2903,194215.7553,200383.4582


In [184]:
death = pd.read_csv('https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/deaths_prediction.csv')

In [185]:
death

,time,dead,dead_low,dead_high
0,7/29/2020,4457.672053,4442.130056,4472.148723
1,7/30/2020,4489.281939,4470.465939,4506.684092
2,7/31/2020,4520.776952,4498.364630,4541.215381
3,8/1/2020,4552.160918,4526.103289,4575.747185
4,8/2/2020,4583.437635,4553.685923,4610.284097
5,8/3/2020,4614.610869,4581.116484,4644.830712
6,8/4/2020,4645.684357,4608.398865,4679.391622


## Get Hazard Score


In [186]:
prediction = pd.merge(case_fit, death, left_on='time', right_on='time')

In [187]:
prediction = prediction.iloc[:,[0,1,4]]

In [188]:
prediction['positive_increase'] = prediction['positive'].diff()
prediction['dead_increase'] = prediction['dead'].diff()

In [189]:
prediction = prediction.iloc[1:,[0,3,4]].reset_index(drop=True)

In [190]:
def norm_hazard(alpha,data):
  alpha = alpha
  data['hazard'] = data['positive_increase']*alpha + data['dead_increase']*(1-alpha)
  return data

In [191]:
pred_haz = norm_hazard(0.012,prediction).iloc[:,[0,-1]]

## Get Risk Score

In [210]:
# To get the risk score for each date
risk_score['{}'.format(pred_haz['time'][0])] = risk_score['confirmed_score']*pred_haz['hazard'][0]
risk_score['{}'.format(pred_haz['time'][1])] = risk_score['confirmed_score']*pred_haz['hazard'][1]
risk_score['{}'.format(pred_haz['time'][2])] = risk_score['confirmed_score']*pred_haz['hazard'][2]
risk_score['{}'.format(pred_haz['time'][3])] = risk_score['confirmed_score']*pred_haz['hazard'][3]
risk_score['{}'.format(pred_haz['time'][4])] = risk_score['confirmed_score']*pred_haz['hazard'][4]
risk_score['{}'.format(pred_haz['time'][5])] = risk_score['confirmed_score']*pred_haz['hazard'][5]

,City,confirmed_score,death_score,7/30/2020,7/31/2020,8/1/2020,8/2/2020,8/3/2020,8/4/2020,weekly_score
0,Acton,0.189272,0.220192,11.511339,11.547813,11.586154,11.626378,11.668499,11.712534,11.608786
1,Agoura Hills,0.181774,0.213950,11.055304,11.090333,11.127155,11.165785,11.206238,11.248528,11.148890
2,Alhambra,0.144250,0.270825,8.773122,8.800920,8.830141,8.860796,8.892898,8.926458,8.847389
3,Altadena,0.187301,0.329177,11.391408,11.427503,11.465444,11.505249,11.546931,11.590507,11.487840
4,Arcadia,0.155135,0.314134,9.435108,9.465004,9.496429,9.529398,9.563922,9.600014,9.514979
...,...,...,...,...,...,...,...,...,...,...
85,Venice,0.341497,0.290481,20.769475,20.835284,20.904461,20.977035,21.053033,21.132483,20.945295
86,West Hills,0.418424,0.205003,25.448053,25.528687,25.613447,25.702369,25.795487,25.892833,25.663479
87,Wilmington,0.234219,0.206903,14.244955,14.290090,14.337536,14.387312,14.439436,14.493927,14.365543
88,Winnetka,0.254805,0.189505,15.496942,15.546045,15.597661,15.651811,15.708517,15.767797,15.628129


### Weekly Risk Score and Level

In [ ]:
# To get the weekly score for each city
risk_score['weekly_score'] = risk_score.iloc[:,3:].mean(axis=1)

In [209]:
# To get the weekly score for each city and store it to a DF
risk_score['weekly_score'] = risk_score.iloc[:,3:].mean(axis=1)
weekly_risk = risk_score.loc[:,['City','weekly_score']]

,City,weekly_score,level
0,Acton,11.608786,2
1,Agoura Hills,11.148890,1
2,Alhambra,8.847389,1
3,Altadena,11.487840,2
4,Arcadia,9.514979,1
...,...,...,...
85,Venice,20.945295,4
86,West Hills,25.663479,4
87,Wilmington,14.365543,2
88,Winnetka,15.628129,3


In [195]:
# Get the weekly level
weekly_level = []
for risk in weekly_risk['weekly_score']:
  if risk <= weekly_risk['weekly_score'].quantile(0.25):
    b = 1
  elif risk <= weekly_risk['weekly_score'].quantile(0.50) and risk >weekly_risk['weekly_score'].quantile(0.25):
    b = 2 
  elif risk <= weekly_risk['weekly_score'].quantile(0.75) and risk > weekly_risk['weekly_score'].quantile(0.50):
    b = 3
  else: b =4
  weekly_level.append(b)

In [211]:
weekly_risk['level'] = weekly_level

,City,weekly_score,level
0,Acton,11.608786,2
1,Agoura Hills,11.148890,1
2,Alhambra,8.847389,1
3,Altadena,11.487840,2
4,Arcadia,9.514979,1
...,...,...,...
85,Venice,20.945295,4
86,West Hills,25.663479,4
87,Wilmington,14.365543,2
88,Winnetka,15.628129,3


### Time Series Risk Score

In [212]:
risk_trunc = risk_score.iloc[:,[0,3,4,5,6,7,8]]

,City,7/30/2020,7/31/2020,8/1/2020,8/2/2020,8/3/2020,8/4/2020
0,Acton,11.511339,11.547813,11.586154,11.626378,11.668499,11.712534
1,Agoura Hills,11.055304,11.090333,11.127155,11.165785,11.206238,11.248528
2,Alhambra,8.773122,8.800920,8.830141,8.860796,8.892898,8.926458
3,Altadena,11.391408,11.427503,11.465444,11.505249,11.546931,11.590507
4,Arcadia,9.435108,9.465004,9.496429,9.529398,9.563922,9.600014
...,...,...,...,...,...,...,...
85,Venice,20.769475,20.835284,20.904461,20.977035,21.053033,21.132483
86,West Hills,25.448053,25.528687,25.613447,25.702369,25.795487,25.892833
87,Wilmington,14.244955,14.290090,14.337536,14.387312,14.439436,14.493927
88,Winnetka,15.496942,15.546045,15.597661,15.651811,15.708517,15.767797


In [225]:
trans = risk_trunc.set_index('City').transpose()

City,Acton,Agoura Hills,Alhambra,Altadena,Arcadia,Artesia,Avalon,Baldwin Park,Bell,Bell Gardens,Bellflower,Beverly Hills,Burbank,Calabasas,Canoga Park,Chatsworth,Claremont,Commerce,Compton,Covina,Cudahy,Culver City,Downey,Duarte,El Monte,El Segundo,Encino,Gardena,Glendale,Glendora,Granada Hills,Harbor City,Hawthorne,Hermosa Beach,Hidden Hills,Huntington Park,Industry,Inglewood,Irwindale,La Canada Flintridge,...,Newhall,North Hills,North Hollywood,Northridge,Pacific Palisades,Pacoima,Palmdale,Panorama City,Paramount,Pico Rivera,Playa Del Rey,Playa Vista,Pomona,Porter Ranch,Rancho Palos Verdes,Redondo Beach,Reseda,Rolling Hills,Rolling Hills Estates,San Dimas,San Fernando,San Pedro,Sherman Oaks,South El Monte,South Gate,South Pasadena,Studio City,Sun Valley,Sunland,Sylmar,Tarzana,Torrance,Tujunga,Valley Village,Van Nuys,Venice,West Hills,Wilmington,Winnetka,Woodland Hills
7/30/2020,11.511339,11.055304,8.773122,11.391408,9.435108,6.105812,15.482879,8.863734,10.855953,12.098596,9.577867,26.283179,18.421529,17.088134,12.511706,19.145580,9.217761,11.572470,11.098441,16.787091,14.267491,16.699071,6.986175,13.953802,13.787092,17.657165,12.443905,9.378554,11.994193,11.497809,17.332401,17.045712,9.531358,13.992398,9.921272,10.464533,17.957258,13.954136,7.839930,15.352202,...,15.210365,9.563546,9.153170,12.085822,25.979491,7.079061,13.902148,11.134542,7.168554,8.990225,27.860301,24.430494,16.415666,20.203615,15.158480,15.353469,17.132571,16.030965,14.541490,15.844695,15.894964,16.965069,11.212587,26.230872,13.949716,25.980099,21.779933,13.708028,18.421392,14.381820,13.016487,17.655065,21.269917,21.224911,7.046091,20.769475,25.448053,14.244955,15.496942,19.735650
7/31/2020,11.547813,11.090333,8.800920,11.427503,9.465004,6.125159,15.531937,8.891819,10.890351,12.136931,9.608215,26.366459,18.479899,17.142279,12.551350,19.206244,9.246968,11.609137,11.133607,16.840282,14.312698,16.751983,7.008311,13.998015,13.830777,17.713112,12.483334,9.408271,12.032197,11.534240,17.387319,17.099722,9.561559,14.036733,9.952708,10.497690,18.014156,13.998350,7.864772,15.400846,...,15.258560,9.593849,9.182172,12.124117,26.061808,7.101492,13.946197,11.169823,7.191268,9.018711,27.948578,24.507903,16.467680,20.267631,15.206511,15.402117,17.186857,16.081759,14.587566,15.894899,15.945328,17.018824,11.248115,26.313985,13.993916,26.062418,21.848943,13.751462,18.479761,14.427389,13.057731,17.711006,21.337312,21.292163,7.068417,20.835284,25.528687,14.290090,15.546045,19.798183
8/1/2020,11.586154,11.127155,8.830141,11.465444,9.496429,6.145495,15.583506,8.921342,10.926509,12.177228,9.640116,26.454000,18.541255,17.199194,12.593023,19.270012,9.277669,11.647682,11.170573,16.896195,14.360219,16.807602,7.031580,14.044491,13.876698,17.771923,12.524781,9.439508,12.072147,11.572536,17.445049,17.156497,9.593305,14.083338,9.985753,10.532544,18.073967,14.044827,7.890884,15.451980,...,15.309221,9.625702,9.212659,12.164371,26.148339,7.125070,13.992501,11.206908,7.215145,9.048655,28.041372,24.589273,16.522356,20.334923,15.256999,15.453255,17.243920,16.135154,14.635999,15.947673,15.998269,17.075330,11.285460,26.401353,14.040378,26.148950,21.921486,13.797120,18.541117,14.475291,13.101085,17.769810,21.408156,21.362857,7.091885,20.904461,25.613447,14.337536,15.597661,19.863917
8/2/2020,11.626378,11.165785,8.860796,11.505249,9.529398,6.166831,15.637607,8.952314,10.964443,12.219503,9.673584,26.545840,18.605625,17.258905,12.636742,19.336912,9.309878,11.688119,11.209354,16.954853,14.410074,16.865953,7.055992,14.093250,13.924874,17.833622,12.568263,9.472279,12.114057,11.612712,17.505612,17.216059,9.626610,14.132231,10.020420,10.569110,18.136714,14.093586,7.918279,15.505624,...,15.362370,9.659120,9.244642,12.206602,26.239118,7.149806,14.041079,11.245815,7.240193,9.080069,28.138723,24.674640,16.579717,20.405520,15.309967,15.506904,17.303786,16.191170,14.686811,16.003039,16.053811,17.134610,11.324640,26.493010,14.089122,26.239732,21.997591,13.845019,18.605486,14.525545,13.146568,17.831501,21.482478

In [226]:
trans_reindex = trans.reset_index()

In [231]:
melted = pd.melt(trans_reindex, id_vars='index', value_vars=trans_reindex.columns[1:])

,index,City,value
0,7/30/2020,Acton,11.511339
1,7/31/2020,Acton,11.547813
2,8/1/2020,Acton,11.586154
3,8/2/2020,Acton,11.626378
4,8/3/2020,Acton,11.668499
...,...,...,...
535,7/31/2020,Woodland Hills,19.798183
536,8/1/2020,Woodland Hills,19.863917
537,8/2/2020,Woodland Hills,19.932878
538,8/3/2020,Woodland Hills,20.005094


In [233]:
sort_melt = melted.sort_values(by='index').reset_index(drop=True).rename(columns={'index':'Date','value':'Score'})

,Date,City,Score
0,7/30/2020,Acton,11.511339
1,7/30/2020,Harbor City,17.045712
2,7/30/2020,Hawthorne,9.531358
3,7/30/2020,Hermosa Beach,13.992398
4,7/30/2020,Van Nuys,7.046091
...,...,...,...
535,8/4/2020,Northridge,12.297058
536,8/4/2020,Pacific Palisades,26.433559
537,8/4/2020,Pacoima,7.202789
538,8/4/2020,Lynwood,16.045218


In [240]:
# Calculate the quantile for each date
q1 = np.quantile(trans,0.25,axis=1).reshape(-1,1)
q2 = np.quantile(trans,0.5,axis=1).reshape(-1,1)
q3 = np.quantile(trans,0.75,axis=1).reshape(-1,1)
quantiles = np.concatenate([q1,q2,q3], axis = 1)
quant_df = pd.DataFrame(quantiles, columns=['q1','q2','q3'], index=trans.index)

,q1,q2,q3
7/30/2020,11.154053,14.461655,17.207113
7/31/2020,11.189396,14.507478,17.261634
8/1/2020,11.226546,14.555645,17.318946
8/2/2020,11.265522,14.606178,17.379072
8/3/2020,11.306336,14.659095,17.442035
8/4/2020,11.349003,14.714415,17.507857


In [242]:
merged_with_quantile = pd.merge(sort_melt, quant_df, left_on='Date', right_on=quant_df.index)

In [ ]:
level = []
for score, q1, q2, q3 in zip(merged_with_quantile['Score'], merged_with_quantile['q1'], merged_with_quantile['q2'], merged_with_quantile['q3']):
  if score <= q1:
    lev = 1
  elif score <= q2 and score > q1:
    lev = 2
  elif score <= q3 and score > q2:
    lev = 3
  else : 
    lev = 4
  level.append(lev)

In [252]:
merged_with_quantile['Daily_Level'] = level

,Date,City,Score,q1,q2,q3,Daily_Level
0,7/30/2020,Acton,11.511339,11.154053,14.461655,17.207113,2
1,7/30/2020,Harbor City,17.045712,11.154053,14.461655,17.207113,3
2,7/30/2020,Hawthorne,9.531358,11.154053,14.461655,17.207113,1
3,7/30/2020,Hermosa Beach,13.992398,11.154053,14.461655,17.207113,2
4,7/30/2020,Van Nuys,7.046091,11.154053,14.461655,17.207113,1
...,...,...,...,...,...,...,...
535,8/4/2020,Northridge,12.297058,11.349003,14.714415,17.507857,2
536,8/4/2020,Pacific Palisades,26.433559,11.349003,14.714415,17.507857,4
537,8/4/2020,Pacoima,7.202789,11.349003,14.714415,17.507857,1
538,8/4/2020,Lynwood,16.045218,11.349003,14.714415,17.507857,3


In [254]:
final_risk = merged_with_quantile.iloc[:,[0,1,2,-1]]

,Date,City,Score,Daily_Level
0,7/30/2020,Acton,11.511339,2
1,7/30/2020,Harbor City,17.045712,3
2,7/30/2020,Hawthorne,9.531358,1
3,7/30/2020,Hermosa Beach,13.992398,2
4,7/30/2020,Van Nuys,7.046091,1
...,...,...,...,...
535,8/4/2020,Northridge,12.297058,2
536,8/4/2020,Pacific Palisades,26.433559,4
537,8/4/2020,Pacoima,7.202789,1
538,8/4/2020,Lynwood,16.045218,3


In [255]:
# final_risk.to_csv('final_risk_score.csv',index=False)